# ALC Simulations based on Analytical Solutions

### $\Delta_1$ resonance

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from scipy.constants import pi
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from scipy.io import loadmat

pio.templates.default = "DemonLab"


def muon_polarization_time_integrated(magnetic_fields, theta, Pz_0, off_resonance_state_rate, A_iso, D_parallel):
    """
    Calculate the time-integrated muon polarization for a given magnetic field and angle theta.
    """
    gamma_muon = 135.5 # in MHz/T
    inv_muon_lifetime = 0.4551 # in MHz
    osc_relaxation = inv_muon_lifetime + off_resonance_state_rate # in MHz

    def calc_q(theta, D_parallel):
        return 0.75 * D_parallel * np.sin(theta) * np.cos(theta)

    def calc_nu_mu(A_iso, D_parallel, theta):
        return (A_iso + D_parallel/2 * (3 * np.cos(theta)**2 - 1)) / 2

    q = calc_q(theta, D_parallel)
    nu_mu = calc_nu_mu(A_iso, D_parallel, theta)
    nu_mu0 = gamma_muon * magnetic_fields

    nominator = 0.5 * q**2 * Pz_0
    denominators = (osc_relaxation/2/pi)**2 + q**2 + (nu_mu0 - nu_mu)**2

    return 1 - nominator/denominators

In [ ]:
# magnetic_fields = np.linspace(1.8675, 1.9325, 3000) # in Tesla
# # magnetic_fields = np.linspace(0, 5, 3000) # in Tesla
# # thetas = np.radians([1, 5, 20, 45, 70, 85, 89]) # given in degrees and converted to radians
# thetas = np.radians(np.linspace(0, 90, 200))
# # thetas = np.radians([45])
# Pz_0 = 1.0
# off_resonance_state_rate = 0
# A_iso = 514.8 # in MHz
# D_parallel = 2 # in MHz

# Parameters for SrTiO3
magnetic_fields = np.linspace(0, 0.4, 3000) # in Tesla
thetas = np.radians([1, 5, 20, 45, 70, 85, 89]) # given in degrees and converted to radians
thetas = np.radians([45])
Pz_0 = 1.0
off_resonance_state_rate = 0
A_iso = 50 # in MHz
D_parallel = 15.5 # in MHz

df = pd.DataFrame({'B / T': magnetic_fields})
for theta in thetas:
    df[f"θ = {np.degrees(theta):.2f}°"] = muon_polarization_time_integrated(magnetic_fields, theta, Pz_0, off_resonance_state_rate, A_iso, D_parallel)

peak_positions = {
    col: magnetic_fields[np.argmin(df[col].values)]
    for col in df.columns if col != "B / T"
}

# print(len(peak_positions))

# savemat('../../MatLab/ALCvsMQ/ana_peak_positions.mat', {
#     "peak_positions": np.array(list(peak_positions.values()))
# })

# Melt for seaborn plotting
df_melted = df.melt(id_vars="B / T", var_name="θ", value_name="Pz")

# Plot
# plt.figure(figsize=(8,5))
# sns.lineplot(data=df_melted, x="B / T", y="Pz", hue="θ")
# plt.xlabel("B / T")
# plt.ylabel(r"$P_z$")
# plt.legend(title="Angle")

fig = px.line(df_melted, x="B / T", y="Pz", color="θ")
fig.update_layout(height=400, width=600, legend=dict(y=0.5, x=1.02, xanchor="left", yanchor="middle"))


### Numerical integration over $\theta$

In [ ]:
magnetic_fields = np.linspace(1.8675, 1.9325, 3000) # in Tesla
Pz_0 = 1.0
off_resonance_state_rate = 0
A_iso = 514.8 # in MHz
D_parallel = 2 # in MHz

thetas = np.radians(np.linspace(0, 90, 500)) # given in degrees and converted to radians

# compute all polarizations into a 2D array: shape = (len(magnetic_fields), len(thetas))
polarization_matrix = np.column_stack([
    muon_polarization_time_integrated(magnetic_fields, theta, Pz_0, off_resonance_state_rate, A_iso, D_parallel)
    for theta in thetas
])

df = pd.DataFrame(polarization_matrix, columns=[f"θ_{i}" for i in range(len(thetas))])
df.insert(0, "B", magnetic_fields)

# weighted mean across thetas
weights = np.sin(thetas)
polarizations_weighted = np.average(polarization_matrix, axis=1, weights=weights)

df["Pz_avg"] = polarizations_weighted

fig = go.Figure()
fig.add_trace(go.Scatter(x=df["B"], y=df["Pz_avg"], mode="lines", line=dict(color='black')))
fig.update_layout(height=400, width=600,
                  yaxis_title=r"$\Large \left < P_z \right >$",
                  xaxis_title="B / T")
# fig.update_layout(showlegend=False)
fig.show(renderer='browser')


### Comparison with numerical simulation

In [ ]:
# num_data = loadmat("C:/Users/walk_d/GitHub/MQ_muSR/MatLab/ALCvsMQ/peak_positions_251021.mat")
num_data = loadmat(r"C:\Users\david\GitHub\MQ_muSR\MatLab\ALCvsMQ\peak_positions.mat")
num_data = pd.DataFrame(num_data["peak_positions"], columns=['theta', 'peak_positions'])
ana_data = pd.DataFrame({
    'theta': peak_positions.keys(),
    'peak_positions': list(peak_positions.values())
})
thetas = np.linspace(0, 90, 200)
thetas = thetas[ana_data['peak_positions'] > 1.88]
ana_data = ana_data[ana_data['peak_positions'] > 1.88]

df = pd.DataFrame({
    'theta': thetas,
    'peak_pos_diff': ana_data['peak_positions'].values - num_data['peak_positions'].values
})

fig = px.line(x=df['theta'], y=df['peak_pos_diff'], labels={'x': 'θ', 'y': 'Peak Position difference (T)'}, title='Peak Position vs θ')
fig.show()

# df.to_csv('../../MatLab/ALCvsMQ/ana_vs_num_peak_positions.csv', index=False)

## Find best parameters for numerical simulation

All .mat files in the Grid Search folder are compared

In [ ]:
import glob
import os

path = r"../../MatLab/ALCvsMQ/Grid Search/*.mat"

fig = go.Figure()

for file in glob.glob(path):
    mat = loadmat(file, squeeze_me=True)
    trace_name = os.path.splitext(os.path.basename(file))[0]
    print(trace_name)

    arr = np.array(mat['peak_positions'])
    print(arr)
    thetas = arr[:, 0]                # degrees
    peaks = arr[:, 1]                 # numerical peak positions
    peak_diffs = peaks - ana_data['peak_positions'].values
    fig.add_trace(go.Scatter(x=thetas, y=peak_diffs, mode="lines", name=trace_name))

fig.update_layout(
    template='DemonLab',
    height=400,
    width=800,
    legend=dict(y=0.5, x=1.02, xanchor="left", yanchor="middle"),
    xaxis_title='θ',
    yaxis_title='Peak Position difference / T',
)
fig.show(renderer='browser')

